## Aqui, faremos análises

In [1]:
%defaultDatasource jdbc:h2:mem:db

Tabela das Receitas

In [2]:
DROP VIEW IF EXISTS Recipes

In [3]:
CREATE VIEW Recipes AS 
SELECT "Recipe ID" AS recipe_id, Cuisine
FROM CSVREAD('../data/raw/01_Recipe_Details.csv');

Tabela dos Ingredientes

In [4]:
DROP TABLE IF EXISTS Ingredients

In [5]:
CREATE VIEW Ingredients AS 
SELECT *
FROM CSVREAD('../data/raw/02_Ingredients.csv');

Tabela de ligação entre receitas e ingredientes

In [6]:
DROP TABLE IF EXISTS Recipes_Ingredients

In [7]:
CREATE VIEW Recipes_Ingredients AS
SELECT "Recipe ID" AS recipe_id, "Entity ID" as ingredient_id
FROM CSVREAD('../data/raw/04_Recipe-Ingredients_Aliases.csv');

Tabela de ligação entre Foodb e ingredientes

In [8]:
DROP TABLE IF EXISTS Determining_Ingredient_Group

In [9]:
CREATE VIEW Determining_Ingredient_Group AS
SELECT *
FROM CSVREAD('../data/interim/Ingredients_to_Foodb_Groups.csv');

Tabela de ligação entre país e região

In [10]:
DROP TABLE IF EXISTS COUNTRIES

In [11]:
CREATE VIEW Countries AS
SELECT Country, "World Region" AS region
FROM CSVREAD('../data/interim/CountriesTable.csv');

Criando tabela do food prices

In [35]:
DROP TABLE IF EXISTS prices

In [36]:
CREATE TABLE Prices AS
SELECT "Country Name" AS country, "Series Name" AS category, "2017 [YR2017]" AS price
FROM CSVREAD('../data/raw/Food_Prices_Data.csv');

Atualizando os valores dos preços para float, e colocando como 0 os que não tem dados

In [38]:
UPDATE Prices
SET price = CASE 
                    WHEN price = '..' THEN 0
                    WHEN price NOT LIKE '%[^0-9.]%' THEN CAST(price AS FLOAT)
                    ELSE price
                END;


Criando

In [49]:
DROP TABLE IF EXISTS Prices_per_Region

In [50]:
CREATE TABLE Prices_per_Region AS 
SELECT DISTINCT C.region, AVG(COALESCE(CAST(price AS FLOAT), 0)) AS average_price, P.category
FROM Countries C, Prices P
WHERE C.Country = P.country 
GROUP BY C.region, P.category
ORDER BY region

In [52]:
CALL CSVWRITE('../data/interim/Category_Prices_per_Region.csv', 'SELECT * FROM Prices_per_Region');

120